<a href="https://colab.research.google.com/github/mint-aguccim/Machine_Learning_Programming/blob/main/5%EC%A3%BC%EC%B0%A8/5%EC%A3%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# -----------------------------
# 1) 데이터 준비
# -----------------------------
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/diabetes.csv")

In [3]:
# 입력(X): Outcome, BMI 제외 / 타깃(y): BMI
X = df.drop(columns=["Outcome", "BMI"])
y = df["BMI"]


In [4]:
# Pima 데이터의 '0'을 결측으로 보고 처리할 피처들
zero_as_nan = ["Glucose", "BloodPressure", "SkinThickness", "Insulin"]
for c in zero_as_nan:
    if c in X.columns:
        X.loc[X[c] == 0, c] = pd.NA

In [5]:
# train/test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [6]:
# -----------------------------
# 2) 딥러닝을 위한 데이터 전처리
# -----------------------------
# 🧠 2-1) 결측치 대치
# 파이프라인 대신 직접 학습/변환을 수행합니다.
imputer = SimpleImputer(strategy="median")
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [7]:
# 🧠 2-2) 피처 스케일링
# 신경망은 피처 스케일에 민감하므로 정규화/표준화가 필수입니다.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

print(f"훈련 데이터 형태: {X_train_scaled.shape}")

훈련 데이터 형태: (614, 7)


In [8]:
# -----------------------------
# 3) 딥러닝 모델 구성 (회귀)
# -----------------------------
model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)), # 입력층
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # 🧠 회귀 문제의 출력층: 뉴런 1개, 활성화 함수 없음
])

In [9]:
# 🧠 모델 컴파일: 회귀 문제에 맞는 손실 함수와 옵티마이저 설정
model.compile(optimizer='adam',
              loss='mean_squared_error', # 대표적인 회귀 손실 함수
              metrics=['mae']) # Mean Absolute Error도 함께 추적

model.summary()

# 🧠 조기 종료 콜백: 과적합을 방지하기 위해 검증 손실이 개선되지 않으면 학습 중단
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,625 (10.25 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# -----------------------------
# 4) 모델 학습
# -----------------------------
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    validation_split=0.2, # 훈련 데이터의 일부를 검증용으로 사용
    callbacks=[early_stopping],
    verbose=0 # 학습 과정 로그는 생략
)

print("\n모델 학습 완료.")


모델 학습 완료.


In [11]:
# -----------------------------
# 5) 모델 평가 (회귀 지표)
# -----------------------------
def report(name, y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_true, y_pred)
    print(f"{name:>14} | MAE={mae:.3f} | RMSE={rmse:.3f} | R²={r2:.3f}")

In [12]:
# 테스트 데이터로 예측
y_pred_dl = model.predict(X_test_scaled).flatten() # 🧠 예측 결과는 (n, 1) 형태이므로 1차원으로 변환

print("\n=== Test Performance ===")
report("Deep Learning", y_test, y_pred_dl)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

=== Test Performance ===
 Deep Learning | MAE=5.497 | RMSE=8.390 | R²=0.010
